# Agentic AI MCP - Server Only

This notebook starts an MCP server that exposes tools.
Run this on the machine where you want to host the tools.

Another machine can connect to this server using the client notebook.

## Step 1: Setup

In [ ]:
# install if needed
# !uv pip install agentic_ai_mcp==0.5.6

import agentic_ai_mcp

print(f"Version: {agentic_ai_mcp.__version__}")

## Step 2: Define Your Functions (to be used as MCP tools)

In [ ]:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b
    
def greet(name: str, times: int = 1) -> str:
    """Greet someone."""
    return ("Hello, " + name + "! ") * times

## Step 3: Create AgenticAIServer and Register Tools

Configure the host/port.

In [ ]:
from agentic_ai_mcp import AgenticAIServer

server = AgenticAIServer(host="0.0.0.0", port=8888)

# register your tools
server.register_tool(add)
server.register_tool(multiply)
server.register_tool(greet)

print(f"Tools: {server.tools}")
print(f"URL: {server.mcp_url}")

## Step 4: Start the MCP Server

This will start the server. The server URL will be:
- Local: `http://127.0.0.1:8888/mcp`
- Remote: `http://<your-ip>:8888/mcp`

Give this URL to the client notebook.

**Note:** The `run()` method runs until interrupted (Ctrl+C or kernel restart).

In [ ]:
# get your IP for remote connections
import socket

hostname = socket.gethostname()
try:
    local_ip = socket.gethostbyname(hostname)
except:
    local_ip = "<your-ip>"

print("\nStarting server...")
print("Local URL:  http://127.0.0.1:8888/mcp")
print(f"Remote URL: http://{local_ip}:8888/mcp")
print("\nPress Ctrl+C or restart kernel to stop.\n")

# Runs until Ctrl+C
server.run()